In [1]:
import pandas as pd

boss = {'HP': 103, "damage": 9, "armor": 2}
player = {"HP": 100, "damage": 4, "armor": 1}

weapons = ["Weapons:    Cost  Damage  Armor",
"Dagger        8     4       0",
"Shortsword   10     5       0",
"Warhammer    25     6       0",
"Longsword    40     7       0",
"Greataxe     74     8       0"]

armor = ["Armor:      Cost  Damage  Armor",
"Leather      13     0       1",
"Chainmail    31     0       2",
"Splintmail   53     0       3",
"Bandedmail   75     0       4",
"Platemail   102     0       5"]

rings = ["Rings:      Cost  Damage  Armor",
"Damage+1    25     1       0",
"Damage+2    50     2       0",
"Damage+3   100     3       0",
"Defense+1   20     0       1",
"Defense+2   40     0       2",
"Defense+3   80     0       3"]

In [2]:
def parse(tsv):
    splits = []
    for row in tsv:
        splits.append(row.split())
    return pd.DataFrame(splits[1:], columns=splits[0]).set_index(splits[0][0])

# - Damage is max(1, attacker_damage - defender_armor)
def attack(p1, p2):
    return max(1, p1["damage"] - p2["armor"])

# - Player goes 1st
def battle(player, boss):
    player["attack"] = attack(player, boss)
    boss["attack"] = attack(boss, player)
    while player["HP"] > 0:
#         print("Player: " + str(player["HP"]) + ", Boss: " + str(boss["HP"]))
        boss["HP"] -= player["attack"]
        if boss["HP"] > 0:
            player["HP"] -= boss["attack"]
        else:
            return "Boss"
    return "Player"

In [3]:
none = pd.DataFrame({"None": {"Cost": "0", "Damage": "0", "Armor": "0"}}).transpose()
weapons = parse(weapons)
armor = pd.concat([parse(armor), none])
rings = pd.concat([parse(rings), none])

In [4]:
import itertools
ring_combos = list(itertools.combinations(rings.index, 2))

In [5]:
# - exactly 1 weapon, 0-1 armor, 0-2 rings
outcomes = []
for w in weapons.index:
    for a in armor.index:
        for r in ring_combos:
            cost = sum(map(int, [weapons.loc[(w, "Cost")], armor.loc[(a, "Cost")], rings.loc[r[0], "Cost"], rings.loc[r[1], "Cost"]]))
            player["damage"] = sum(map(int, [weapons.loc[(w, "Damage")], armor.loc[(a, "Damage")], rings.loc[r[0], "Damage"], rings.loc[r[1], "Damage"]]))
            player["armor"] = sum(map(int, [weapons.loc[(w, "Armor")], armor.loc[(a, "Armor")], rings.loc[r[0], "Armor"], rings.loc[r[1], "Armor"]]))
            player["HP"] = 100
            boss["HP"] = 103
            
            outcomes.append([cost, w, a, r, battle(player, boss)])

In [6]:
victories = [x for x in outcomes if x[-1] == "Boss"]
victories.sort()
victories[0]

[121, 'Longsword', 'Chainmail', ('Damage+2', 'None'), 'Boss']

In [7]:
losses = [x for x in outcomes if x[-1] == "Player"]
losses.sort(reverse=True)
losses[0]

[201, 'Dagger', 'Leather', ('Damage+3', 'Defense+3'), 'Player']

In [ ]:
boss = {"HP": 71, "Armor": 0, "Damage": 10, "Mana": 250}
player = {"HP": 50, "Armor": 0, "Mana": 500}

spells = {"Magic Missile": {"Cost": 53, "Damage": 4},
         "Drain": {"Cost": 73, "Damage": 2, "HP": 2},
         "Shield": {"Cost": 113, "Duration": 6, "Armor": 7},
         "Poison": {"Cost": 173, "Duration": 6, "Damage": 3},
         "Recharge": {"Cost": 229, "Duration": 5, "Mana": 101}}